### Function versions 08-05-23

In [ ]:
def get_mean_NDVI(point_of_interest_file, ndvi_raster_file, buffer_type=None, buffer_dist=None, network_file=None, network_type=None,
                  trip_time=None, travel_speed=None, output_dir=os.getcwd()):
    # Read and process user input, check conditions
    poi = gpd.read_file(point_of_interest_file)
    if all(poi['geometry'].geom_type == 'Point') or all(poi['geometry'].geom_type == 'Polygon'):
        geom_type = poi.iloc[0]['geometry'].geom_type
    else:
        raise ValueError("Please make sure all geometries are of 'Point' type or all geometries are of 'Polygon' type and re-run the function")

    if not poi.crs.is_projected:
        raise ValueError("The CRS of the PoI dataset is currently geographic, please transform it into a local projected CRS and re-run the function")
    else:
        epsg = poi.crs.to_epsg()

    if poi['id'].isnull().values.any():
        poi['id'] = poi['id'].fillna(pd.Series(range(1, len(poi) + 1))).astype(int)

    ndvi_src = rioxarray.open_rasterio(ndvi_raster_file)
    if not ndvi_src.rio.crs.to_epsg() == epsg:
                    print("Adjusting CRS of NDVI file to match with Point of Interest CRS...")
                    ndvi_src.rio.write_crs(f'EPSG:{epsg}', inplace=True)
                    print("Done")

    # Make sure all points of interest are within or do at least intersect (in case of polygons) the NDVI raster provided
    if not all(geom.within(sg.box(*ndvi_src.rio.bounds())) for geom in poi['geometry']):
        if geom_type == "Point":
            raise ValueError("Not all points of interest are within the NDVI file provided, please make sure they are and re-run the function")
        else:
            if not all(geom.intersects(sg.box(*ndvi_src.rio.bounds())) for geom in poi['geometry']):
                raise ValueError("Not all polygons of interest are within, or do at least partly intersect, with the area covered by the NDVI file provided, please make sure they are/do and re-run the function")
            else:
                warnings.warn("Not all polygons of interest are completely within the area covered by the NDVI file provided, results will be based on intersecting part of polygons involved") 

    # Create buffers (or not) based on user input        
    if geom_type == "Point":
        # Make sure all points of interest are within the NDVI raster provided
        if not all(geom.within(sg.box(*ndvi_src.rio.bounds())) for geom in poi['geometry']):
            raise ValueError("Not all points of interest are within the NDVI file provided, please make sure they are and re-run the function")
    
        if buffer_type not in ["euclidian", "network"]:
            raise ValueError("Please make sure that the buffer_type argument is set to either 'euclidian' or 'network' and re-run the function")
    
        if buffer_type == "euclidian":
            if not isinstance(buffer_dist, int) or (not buffer_dist > 0):
                raise TypeError("Please make sure that the buffer distance is set as a positive integer")             

            aoi_gdf = gpd.GeoDataFrame(geometry=poi['geometry'].buffer(buffer_dist))
        else:
            if network_file is not None and (not os.path.splitext(network_file)[1] == ".gpkg"):
                raise ValueError("Please provide the network file in '.gpkg' format")
            elif network_file is not None and (os.path.splitext(network_file)[1] == ".gpkg"):
                network = gpd.read_file(network_file, layer='edges')
            else:
                network = None

            if network is None:
                if not isinstance(buffer_dist, int) or (not buffer_dist > 0):
                    raise TypeError("Please make sure that the buffer distance is set as a positive integer")

                if network_type not in ["walk", "bike", "drive", "all"]:
                    raise ValueError("Please make sure that the network_type argument is set to either 'walk', 'bike, 'drive' or 'all', and re-run the function")
                
                if not isinstance(travel_speed, int) or (not travel_speed > 0):
                    raise TypeError("Please make sure that the travel speed is set as a positive integer")

                if not isinstance(trip_time, int) or (not trip_time > 0):
                    raise TypeError("Please make sure that the trip time is set as a positive integer")             
        
                meters_per_minute = travel_speed * 1000 / 60  # km per hour to m per minute
                epsg_transformer = pyproj.Transformer.from_crs(f"epsg:{epsg}", "epsg:4326")

                # Transform the geometry column to lat-lon coordinates
                aoi_geometry = []
                for geom in poi['geometry']:
                    latlon = epsg_transformer.transform(geom.x, geom.y) # Transform point geometry into latlon for OSMnx
                    graph = ox.graph_from_point(latlon, network_type=network_type, dist=buffer_dist) # Retrieve street network for desired network type and buffer distance surrounding poi
                    nodes = ox.graph_to_gdfs(graph, edges=False) # Create geodataframe which contains only the nodes of the street network
                    x, y = nodes["geometry"].unary_union.centroid.xy # Calculate centroid of node geometries 
                    center_node = ox.distance.nearest_nodes(graph, x[0], y[0]) # Find node which is closest to centroid as base for next steps
                    graph = ox.project_graph(graph) # Project street network graph back to original poi CRS
                    graph_epsg = graph.graph['crs'].to_epsg() # Store EPSG code used for previous step
                    for _, _, _, data in graph.edges(data=True, keys=True): # Calculate the time it takes to cover each edge's distance
                        data["time"] = data["length"] / meters_per_minute
                    isochrone_poly = make_iso_poly(graph, center_node=center_node, trip_time=trip_time) # See separate function for line by line explanation
                    aoi_geometry.append(isochrone_poly)

                aoi_gdf = gpd.GeoDataFrame(geometry=aoi_geometry, crs=f"EPSG:{graph_epsg}").to_crs(f"EPSG:{epsg}")    
            else:
                if not network.crs.to_epsg() == epsg:
                    print("Adjusting CRS of Network file to match with Point of Interest CRS...")
                    network.to_crs(f'EPSG:{epsg}', inplace=True)
                    print("Done")

                # Create bounding box for network file
                bbox_network = network.unary_union.convex_hull

                if not all(geom.within(bbox_network) for geom in poi['geometry']):
                     raise ValueError("Not all points of interest are within the network file provided, please make sure they are and re-run the function")

                aoi_gdf = gpd.GeoDataFrame(geometry=[bbox_network], crs=f'EPSG:{epsg}')
    else:
        aoi_gdf = gpd.GeoDataFrame(geometry=poi['geometry'])

    # Calculate mean NDVI values
    poi['mean_NDVI'] = aoi_gdf.apply(lambda row: ndvi_src.rio.clip([row.geometry]).clip(min=0).mean().values.round(3), axis=1)

    print("Writing results to new geopackage file in specified directory...")
    input_filename, ext = os.path.splitext(point_of_interest_file)
    poi.to_file(os.path.join(output_dir, f"{input_filename}_ndvi_added.gpkg"), driver="GPKG")
    print("Done")
    
    return poi

In [ ]:
def get_landcover_percentages(point_of_interest_file, landcover_raster_file, buffer_type=None, buffer_dist=None, network_file=None, 
                              network_type=None, trip_time=None, travel_speed=None, output_dir=os.getcwd()):
    # Read and process user input, check conditions
    poi = gpd.read_file(point_of_interest_file)
    if all(poi['geometry'].geom_type == 'Point') or all(poi['geometry'].geom_type == 'Polygon'):
        geom_type = poi.iloc[0]['geometry'].geom_type
    else:
        raise ValueError("Please make sure all geometries are of 'Point' type or all geometries are of 'Polygon' type and re-run the function")

    if not poi.crs.is_projected:
        raise ValueError("The CRS of the PoI dataset is currently geographic, please transform it into a local projected CRS and re-run the function")
    else:
        epsg = poi.crs.to_epsg()

    if poi['id'].isnull().values.any():
        poi['id'] = poi['id'].fillna(pd.Series(range(1, len(poi) + 1))).astype(int)

    landcover_src = rioxarray.open_rasterio(landcover_raster_file)
    if not landcover_src.rio.crs.to_epsg() == epsg:
                    print("Adjusting CRS of Land Cover file to match with Point of Interest CRS...")
                    landcover_src.rio.write_crs(f'EPSG:{epsg}', inplace=True)
                    print("Done")

    # Make sure all points of interest are within or do at least intersect (in case of polygons) the NDVI raster provided
    if not all(geom.within(sg.box(*landcover_src.rio.bounds())) for geom in poi['geometry']):
        if geom_type == "Point":
            raise ValueError("Not all points of interest are within the Land Cover file provided, please make sure they are and re-run the function")
        else:
            if not all(geom.intersects(sg.box(*landcover_src.rio.bounds())) for geom in poi['geometry']):
                raise ValueError("Not all polygons of interest are within, or do at least partly intersect, with the area covered by the Land Cover file provided, please make sure they are/do and re-run the function")
            else:
                warnings.warn("Not all polygons of interest are completely within the area covered by the Land Cover file provided, results will be based on intersecting part of polygons involved")

    # Create buffers (or not) based on user input        
    if geom_type == "Point":
        # Make sure all points of interest are within the NDVI raster provided
        if not all(geom.within(sg.box(*landcover_src.rio.bounds())) for geom in poi['geometry']):
            raise ValueError("Not all points of interest are within the Land Cover file provided, please make sure they are and re-run the function")

        if buffer_type not in ["euclidian", "network"]:
            raise ValueError("Please make sure that the buffer_type argument is set to either 'euclidian' or 'network' and re-run the function")
    
        if buffer_type == "euclidian":
            if not isinstance(buffer_dist, int) or (not buffer_dist > 0):
                raise TypeError("Please make sure that the buffer distance is set as a positive integer")             

            aoi_gdf = gpd.GeoDataFrame(geometry=poi['geometry'].buffer(buffer_dist))
        else:
            if network_file is not None and (not os.path.splitext(network_file)[1] == ".gpkg"):
                raise ValueError("Please provide the network file in '.gpkg' format")
            elif network_file is not None and (os.path.splitext(network_file)[1] == ".gpkg"):
                network = gpd.read_file(network_file, layer='edges')
            else:
                network = None

            if network is None:
                if not isinstance(buffer_dist, int) or (not buffer_dist > 0):
                    raise TypeError("Please make sure that the buffer distance is set as a positive integer")

                if network_type not in ["walk", "bike", "drive", "all"]:
                    raise ValueError("Please make sure that the network_type argument is set to either 'walk', 'bike, 'drive' or 'all', and re-run the function")
                
                if not isinstance(travel_speed, int) or (not travel_speed > 0):
                    raise TypeError("Please make sure that the travel speed is set as a positive integer")

                if not isinstance(trip_time, int) or (not trip_time > 0):
                    raise TypeError("Please make sure that the trip time is set as a positive integer")             
        
                meters_per_minute = travel_speed * 1000 / 60  # km per hour to m per minute
                epsg_transformer = pyproj.Transformer.from_crs(f"epsg:{epsg}", "epsg:4326")

                # Transform the geometry column to lat-lon coordinates
                aoi_geometry = []
                for geom in poi['geometry']:
                    latlon = epsg_transformer.transform(geom.x, geom.y) # Transform point geometry into latlon for OSMnx
                    graph = ox.graph_from_point(latlon, network_type=network_type, dist=buffer_dist) # Retrieve street network for desired network type and buffer distance surrounding poi
                    nodes = ox.graph_to_gdfs(graph, edges=False) # Create geodataframe which contains only the nodes of the street network
                    x, y = nodes["geometry"].unary_union.centroid.xy # Calculate centroid of node geometries 
                    center_node = ox.distance.nearest_nodes(graph, x[0], y[0]) # Find node which is closest to centroid as base for next steps
                    graph = ox.project_graph(graph) # Project street network graph back to original poi CRS
                    graph_epsg = graph.graph['crs'].to_epsg() # Store EPSG code used for previous step
                    for _, _, _, data in graph.edges(data=True, keys=True): # Calculate the time it takes to cover each edge's distance
                        data["time"] = data["length"] / meters_per_minute
                    isochrone_poly = make_iso_poly(graph, center_node=center_node, trip_time=trip_time) # See separate function for line by line explanation
                    aoi_geometry.append(isochrone_poly)

                aoi_gdf = gpd.GeoDataFrame(geometry=aoi_geometry, crs=f"EPSG:{graph_epsg}").to_crs(f"EPSG:{epsg}")    
            else:
                if not network.crs.to_epsg() == epsg:
                    print("Adjusting CRS of Network file to match with Point of Interest CRS...")
                    network.to_crs(f'EPSG:{epsg}', inplace=True)
                    print("Done")

                # Create bounding box for network file
                bbox_network = network.unary_union.convex_hull

                if not all(geom.within(bbox_network) for geom in poi['geometry']):
                     raise ValueError("Not all points of interest are within the network file provided, please make sure they are and re-run the function")

                aoi_gdf = gpd.GeoDataFrame(geometry=[bbox_network], crs=f'EPSG:{epsg}')
    else:
        aoi_gdf = gpd.GeoDataFrame(geometry=poi['geometry'])

    # apply the function to each geometry in the GeoDataFrame and create a new Pandas Series
    landcover_percentages_series = aoi_gdf.geometry.apply(lambda x: pd.Series(calculate_landcover_percentages(landcover_src=landcover_src, geometry=x)))
    # rename the columns with the land cover class values
    landcover_percentages_series.columns = ["class_" + str(col) for col in landcover_percentages_series.columns]
    # concatenate the new series to the original dataframe
    poi = pd.concat([poi, landcover_percentages_series], axis=1)

    print("Writing results to new geopackage file in specified directory...")
    input_filename, ext = os.path.splitext(point_of_interest_file)
    poi.to_file(os.path.join(output_dir, f"{input_filename}_LCperc_added.gpkg"), driver="GPKG")
    print("Done")

    return poi

# Function version 15-05-23

def calculate_shortest_distance(geom=None, buffer_dist=None, network_graph=None, park_src=None):   
    park_buffer = park_src.clip(geom.buffer(buffer_dist))

    # get nearest node to house location
    nearest_node = ox.distance.nearest_nodes(network_graph, geom.x, geom.y)
    # Create dictionary to extract geometries for nodes of interest later on
    pos = {n: (network_graph.nodes[n]['x'], network_graph.nodes[n]['y']) for n in network_graph.nodes}
    # get nodes within 25 meters of clipped park polygon boundaries
    boundary_nodes = []
    for boundary in park_buffer.boundary:
        for node in network_graph.nodes():
            node_pos = sg.Point(pos[node])
            if node_pos.distance(boundary) < 25:
                boundary_nodes.append(node)

    # calculate network distances from house location's nearest node to nodes closest to park boundaries within buffer distance from house
    distances = {}
    for node in boundary_nodes:
        try:
            path = nx.shortest_path(network_graph, nearest_node, node, weight='length') 
            distance = sum([network_graph.edges[path[i], path[i+1], 0]['length'] for i in range(len(path)-1)])
            distances[node] = distance
        except:
            pass

    # get minimum distance
    if distances:
        min_distance = min(distances.values())
    else:
        min_distance = np.nan
    
    return min_distance

In [ ]:
def get_shortest_distance_park(point_of_interest_file, crs_epsg=None, buffer_dist=500, park_vector_file=None, network_file=None, network_type=None, 
                               output_dir=os.getcwd()):
    # Read and process user input, check conditions
    poi = gpd.read_file(point_of_interest_file)
    if all(poi['geometry'].geom_type == 'Point') or all(poi['geometry'].geom_type == 'Polygon'):
        geom_type = poi.iloc[0]['geometry'].geom_type
    else:
        raise ValueError("Please make sure all geometries are of 'Point' type or all geometries are of 'Polygon' type and re-run the function")

    if not poi.crs.is_projected:
        if crs_epsg is None:
            print("Warning: The CRS of the PoI dataset is currently geographic, therefore it will now be projected to CRS with EPSG:3395")
            epsg = 3395
            poi.to_crs(f"EPSG:{epsg}", inplace=True)
        else:
            print(f"Warning: The CRS of the PoI dataset is currently geographic, therefore it will now be projected to EPSG:{crs_epsg} as specified")
            epsg = crs_epsg
            poi.to_crs(f"EPSG:{epsg}", inplace=True)
    else:
        epsg = poi.crs.to_epsg()

    # In case of house polygons, transform to centroids
    if geom_type == "Polygon":
        print("Changing geometry type to Point by computing polygon centroids so that network distance can be retrieved...")
        poi['geometry'] = poi['geometry'].centroid
        print("Done \n")

    if "id" in poi.columns:
        if poi['id'].isnull().values.any():
            poi['id'] = poi['id'].fillna(pd.Series(range(1, len(poi) + 1))).astype(int)
    else:
        poi['id'] = pd.Series(range(1, len(poi) + 1)).astype(int)

    if not isinstance(buffer_dist, int) or (not buffer_dist > 0):
        raise TypeError("Please make sure that the buffer distance is set as a positive integer")

    epsg_transformer = pyproj.Transformer.from_crs(f"epsg:{epsg}", "epsg:4326") # EPSG transformer to use for OSM            
    # Read park polygons, retrieve from OSM if not provided by user 
    if park_vector_file is not None:
        park_src = gpd.read_file(park_vector_file)
        if not park_src.crs.to_epsg() == epsg:
            print("Adjusting CRS of Park file to match with Point of Interest CRS...")
            park_src.to_crs(f'EPSG:{epsg}', inplace=True)
            print("Done \n")
    else:
        print("Retrieving parks within buffer distance for point(s) of interest...")
        park_tags = {'leisure': 'park', 'boundary': 'national_park', 'landuse': 'recreation_ground'}
        park_src = gpd.GeoDataFrame()
        for geom in poi['geometry']:
            latlon = epsg_transformer.transform(geom.x, geom.y)
            park_geom = ox.geometries_from_point(latlon, tags=park_tags, dist=buffer_dist)
            park_src = gpd.GeoDataFrame(pd.concat([park_src, park_geom], ignore_index=True), crs=park_geom.crs)
        park_src.to_crs(f"EPSG:{epsg}", inplace=True)
        print("Done \n")

    # Read road network, retrieve from OSM if not provided by user 
    if network_file is not None:
        if os.path.splitext(network_file)[1] not in [".gpkg", ".shp"]:
            raise ValueError("Please provide the network file in '.gpkg' or '.shp' format")
        elif network_file is not None and (os.path.splitext(network_file)[1] == ".gpkg"):
            network = gpd.read_file(network_file, layer='edges')
        elif network_file is not None and (os.path.splitext(network_file)[1] == ".shp"):
            network = gpd.read_file(network_file)

        if not network.crs.to_epsg() == epsg:
            print("Adjusting CRS of Network file to match with Point of Interest CRS...")
            network.to_crs(f'EPSG:{epsg}', inplace=True)
            print("Done \n")

        # Check if house locations are within network file provided
        bbox_network = network.unary_union.envelope
        if not all(geom.within(bbox_network) for geom in poi['geometry']):
            raise ValueError("Not all points of interest are within the network file provided, please make sure they are and re-run the function")

        # Convert network to graph object using momempy
        network_graph = momepy.gdf_to_nx(network)
    else:
        if network_type not in ["walk", "bike", "drive", "all"]:
            raise ValueError("Please make sure that the network_type argument is set to either 'walk', 'bike, 'drive' or 'all', and re-run the function")

        print("Retrieving infrastructure network within buffer distance for point(s) of interest...")
        graph_list = []
        for geom in poi['geometry']:
            latlon = epsg_transformer.transform(geom.x, geom.y)
            network = ox.graph_from_point(latlon, dist=buffer_dist, network_type=network_type)
            network = ox.project_graph(network, to_crs=f"EPSG:{epsg}")
            graph_list.append(network)

        network_graph = nx.MultiDiGraph()
        for graph in graph_list:
            network_graph = nx.union(network_graph, graph)
        print("Done \n")
    
    print("Calculating shortest distances...")
    poi['shortest_distance_park'] = poi.geometry.apply(lambda x: calculate_shortest_distance(geom=x, buffer_dist=buffer_dist, network_graph=network_graph, park_src=park_src))
    print("Done \n")
    
    print("Writing results to new geopackage file in specified directory...")
    input_filename, _ = os.path.splitext(os.path.basename(point_of_interest_file))
    poi.to_file(os.path.join(output_dir, f"{input_filename}_ShortDistPark_added.gpkg"), driver="GPKG")
    print("Done")

    return poi

In [ ]:
if destination == "centroids":
        park_src['centroid'] = park_src['geometry'].centroid
        park_dest = park_src[['centroid']].copy(deep=True)
        park_dest.rename(columns = {'centroid': 'geometry'}, inplace = True)
        park_dest['park_id'] = list(range(len(green_space)))
else:
    entrance = []
    park_src['buffer_20m'] = park_src['geometry'].buffer(20)
    for i in range(len(park_src)):
        intersection = park_src['buffer_20m'][i].boundary.intersection(edges['geometry'])
        intersection = intersection[~intersection.is_empty]
        for point in intersection:
            dic = {'geometry': point, 'park_id': i}
            entrance.append(dic)
    park_dest = gpd.GeoDataFrame(entrance)

# Function version 22-05-23

def calculate_shortest_distance(df_row=None, target_dist=None, network_graph=None, park_src=None, destination=None):   
    ### Step 1: Clip park boundaries to poi incl. buffer to minimize possible destination points
    park_src_buffer = park_src.clip(df_row['geometry'].buffer(target_dist))
    
    ### Step 2: Retrieve nearest network node for house location and calculate euclidian distance between these points
    # Euclidian distance will be added to network distance to minimize distance error
    nearest_node = ox.distance.nearest_nodes(network_graph, df_row['geometry'].x, df_row['geometry'].y)
    penalty_home = df_row['geometry'].distance(sg.Point(network_graph.nodes[nearest_node]['x'], network_graph.nodes[nearest_node]['y']))

    ### Step 3: Create subgraph to only consider network in point's proximity -- save time 
    subgraph = nx.ego_graph(network_graph, nearest_node, radius=target_dist*2, distance="length")

    ### Step 4: Determine park destination points
    pos = {n: (subgraph.nodes[n]['x'], subgraph.nodes[n]['y']) for n in subgraph.nodes} # Create dictionary to extract geometries for nodes of interest
    
    # For each park, retrieve the network nodes which are within 20m of the park boundary or closest to park centroids
    park_nodes = {}
    if destination == "entrance":
        for park_id, geom in zip(park_src_buffer['park_id'], park_src_buffer['geometry']):
            boundary_nodes = [node for node in subgraph.nodes() if sg.Point(pos[node]).distance(geom.boundary) < 20]
            park_nodes[park_id] = boundary_nodes
    else:
        park_nodes = {park_id: [ox.distance.nearest_nodes(subgraph, geometry.x, geometry.y)] for park_id, geometry in zip(park_src_buffer['park_id'], park_src_buffer['geometry'])}
    
    ### Step 5: Calculate the network distances between the house location's nearest node and the park destination points
    # Add penalty_home as defined before to network distance
    distances = {}
    for park_id, nodes in park_nodes.items():
        for node in nodes:
            try:
                path = nx.shortest_path(subgraph, nearest_node, node, weight='length')
                distance = sum([subgraph.edges[path[i], path[i+1], 0]['length'] for i in range(len(path)-1)]) + penalty_home
                distances[node] = distance
            except:
                pass

    # Get the minimum distance (house location to park)
    if distances:
        min_distance = round(min(distances.values()),0)
    else:
        min_distance = np.nan
    
    ### Step 6: Define result, if minimum distance smaller than/equal to target distance threshold --> Good
    if min_distance <= target_dist:
        outcome = "True"
    else:
        outcome = "False"
    
    return outcome, min_distance

In [ ]:
def get_shortest_distance_park(point_of_interest_file, crs_epsg=None, target_dist=300, park_vector_file=None, destination="centroids", 
                               network_file=None, network_type=None, output_dir=os.getcwd()):
    ### Step 1: Read and process user inputs, check conditions
    poi = gpd.read_file(point_of_interest_file)
    if all(poi['geometry'].geom_type == 'Point') or all(poi['geometry'].geom_type == 'Polygon'):
        geom_type = poi.iloc[0]['geometry'].geom_type
    else:
        raise ValueError("Please make sure all geometries are of 'Point' type or all geometries are of 'Polygon' type and re-run the function")

    if not poi.crs.is_projected:
        if crs_epsg is None:
            print("Warning: The CRS of the PoI dataset is currently geographic, therefore it will now be projected to CRS with EPSG:3395")
            epsg = 3395
            poi.to_crs(f"EPSG:{epsg}", inplace=True)
        else:
            print(f"Warning: The CRS of the PoI dataset is currently geographic, therefore it will now be projected to EPSG:{crs_epsg} as specified")
            epsg = crs_epsg
            poi.to_crs(f"EPSG:{epsg}", inplace=True)
    else:
        epsg = poi.crs.to_epsg()

    # In case of house polygons, transform to centroids
    if geom_type == "Polygon":
        print("Changing geometry type to Point by computing polygon centroids so that network distance can be retrieved...")
        poi['geometry'] = poi['geometry'].centroid
        print("Done \n")

    if "id" in poi.columns:
        if poi['id'].isnull().values.any():
            poi['id'] = poi['id'].fillna(pd.Series(range(1, len(poi) + 1))).astype(int)
    else:
        poi['id'] = pd.Series(range(1, len(poi) + 1)).astype(int)

    if not isinstance(target_dist, int) or (not target_dist > 0):
        raise TypeError("Please make sure that the target distance is set as a positive integer")
    
    if destination not in ["centroids", "entrance"]:
        raise TypeError("Please make sure that the destination argument is set to either 'centroids' or 'entrance'")

    ### Step 2: Obtain bounding box in which all points of interest are located, including 1000m buffer to account for edge effects
    poi_polygon = sg.box(*poi.total_bounds).buffer(1000)
    polygon_gdf_wgs = gpd.GeoDataFrame(geometry=[poi_polygon], crs=f"EPSG:{epsg}").to_crs("EPSG:4326") # Transform to 4326 for OSM
    wgs_polygon = polygon_gdf_wgs['geometry'].values[0] # Extract polygon in EPSG 4326

    ### Step 3: Read park polygons, retrieve from OSM if not provided by user 
    if park_vector_file is not None:
        park_src = gpd.read_file(park_vector_file)
        if not park_src.crs.to_epsg() == epsg:
            print("Adjusting CRS of Park file to match with Point of Interest CRS...")
            park_src.to_crs(f'EPSG:{epsg}', inplace=True)
            print("Done \n")
    else:
        print("Retrieving parks within total bounds of point(s) of interest, extended by a 1000m buffer to account for edge effects...")
        park_tags = {'leisure': 'park', 'boundary': 'national_park', 'landuse': 'recreation_ground'}
        park_src = ox.geometries_from_polygon(wgs_polygon, tags=park_tags)
        park_src.to_crs(f"EPSG:{epsg}", inplace=True)
        print("Done \n")
    
    if destination == "centroids":
        park_src['geometry'] = park_src['geometry'].centroid
    
    park_src['park_id'] = list(range(len(park_src)))

    ### Step 3: Read network, retrieve from OSM if not provided by user 
    if network_file is not None:
        if os.path.splitext(network_file)[1] not in [".gpkg", ".shp"]:
            raise ValueError("Please provide the network file in '.gpkg' or '.shp' format")
        elif network_file is not None and (os.path.splitext(network_file)[1] == ".gpkg"):
            network = gpd.read_file(network_file, layer='edges')
        elif network_file is not None and (os.path.splitext(network_file)[1] == ".shp"):
            network = gpd.read_file(network_file)

        if not network.crs.to_epsg() == epsg:
            print("Adjusting CRS of Network file to match with Point of Interest CRS...")
            network.to_crs(f'EPSG:{epsg}', inplace=True)
            print("Done \n")

        # Check if house locations are within network file provided
        bbox_network = network.unary_union.envelope
        if not all(geom.within(bbox_network) for geom in poi['geometry']):
            raise ValueError("Not all points of interest are within the network file provided, please make sure they are and re-run the function")

        # Convert network to graph object using momempy
        network_graph = momepy.gdf_to_nx(network)
    else:
        if network_type not in ["walk", "bike", "drive", "all"]:
            raise ValueError("Please make sure that the network_type argument is set to either 'walk', 'bike, 'drive' or 'all', and re-run the function")
            
        print("Retrieving infrastructure network within total bounds of point(s) of interest, extended by a 1000m buffer to account for edge effects...")
        network_graph = ox.graph_from_polygon(wgs_polygon, network_type=network_type)
        network_graph = ox.project_graph(network_graph, to_crs=f"EPSG:{epsg}")
        print("Done \n")
    
    ### Step 4: Perform calculations and write results to file
    print("Calculating shortest distances...")
    poi[[f'park_within_{target_dist}m', 'distance_to_park']] = poi.apply(lambda row: pd.Series(calculate_shortest_distance(df_row=row, target_dist=target_dist, network_graph=network_graph, park_src=park_src, destination=destination)), axis=1)
    print("Done \n")
    
    print("Writing results to new geopackage file in specified directory...")
    input_filename, _ = os.path.splitext(os.path.basename(point_of_interest_file))
    poi.to_file(os.path.join(output_dir, f"{input_filename}_ShortDistPark_added.gpkg"), driver="GPKG")
    print("Done")

    return poi